# Face Detection and Recognition

## Loading Libraries

In [1]:
!which python
import sys
sys.version

# !python3 -c "import tensorflow as tf; print(tf.__version__)"
# !pip3 install opencv-python
# !pip3 install sklearn
# !pip3 install keras

# !python3 -m pip install Pillow
# from keras.preprocessing import image
# from PIL import Image
# from IPython.display import display, Image

# from IPython.display import display 
# from PIL import Image as image

/usr/bin/python


'3.6.5 (default, Apr  1 2018, 05:46:30) \n[GCC 7.3.0]'

In [2]:
## Loading Libraries
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Dense, Activation, Flatten
from keras.layers import Input
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time


Using TensorFlow backend.


## Face Detection

### Testing imagenet

In [ ]:
img_width, img_height = 224, 224
model = VGG16(weights = "imagenet" , include_top = True , input_shape=(img_width ,img_height ,3) )
# img_path = "/home/Downloads/demo.jpg"
img_path = "lfw_smaller/Aaron_Sorkin/Aaron_Sorkin_0001.jpg"
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print(x.shape)
result = model.predict(x)
print(model.summary())
img_input = Input(shape=(224, 224, 3))

#print decode_predictions(result)


In [ ]:
from keras.applications.resnet50 import decode_predictions
# print(decode_predictions(result))
print('Predicted:', decode_predictions(result, top=3)[0])

### Testing VCG16

In [ ]:
model = VGG16(input_tensor=img_input, include_top=True,weights='imagenet')
last_layer = model.get_layer("fc2").output
out = Dense(units = 135, activation='softmax', name='output')(last_layer)
# print out 
custom_vgg_model = Model(img_input, out)

In [ ]:
for i in custom_vgg_model.layers[:-1]:
    i.trainable = False
custom_vgg_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# training_set = train_datagen.flow_from_directory('/home/train_data',
training_set = train_datagen.flow_from_directory('lfw_smaller',
                                                 target_size = (224, 224),
#                                                  target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
print(training_set.image_shape)

# test_set = test_datagen.flow_from_directory('/home/train_data',
test_set = test_datagen.flow_from_directory('lfw_smaller',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')
print(test_set.image_shape)

In [ ]:
'''
custom_vgg_model.fit_generator(generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, 
validation_data=None, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, 
use_multiprocessing=False, shuffle=True, initial_epoch=0)
'''
custom_vgg_model.fit_generator(training_set,
                               steps_per_epoch=None, epochs=1, verbose=1,
                               validation_data=None, validation_steps=None)
#                          steps_per_epoch = 4154,
#                          epochs = 25,
#                          validation_data = test_set,
#                          validation_steps = 4154
#                         )

In [ ]:
custom_vgg_model.predict(img_input)

# ---------------------------Aman Code End

### Testing SSD

# Face Detection

## HOG & CNN based  Face Detection
https://www.arunponnusamy.com/cnn-face-detector-dlib.html

A Pre Trained Model is used with CNN Detector
http://dlib.net/cnn_face_detector.py.html

In [ ]:
'''
Based On Static image based Detection
'''

#https://askubuntu.com/questions/824834/how-to-install-dlib-for-python3-in-ubuntu-14-04
# !pip3 install dlib

# import required packages
import cv2
import dlib
import argparse
import time
# pip install opencv-python dlib argparse time

# handle command line arguments
# ap = argparse.ArgumentParser()
# ap.add_argument('-i', '--image', required=True, help='path to image file')
# ap.add_argument('-w', '--weights', default='./mmod_human_face_detector.dat', help='path to weights file')

# ap.add_argument('-i', '--image', required=True, help='path to image file')
# ap.add_argument('-w', '--weights', default='./mmod_human_face_detector.dat', help='path to weights file')
# args = ap.parse_args()

image_path = 'training-data/Aaron_Sorkin/Aaron_Sorkin_0001.jpg'
weight_path = 'mmod_human_face_detector.dat'



#### Initialization

# load input image
# image = cv2.imread(args.image)
image = cv2.imread(image_path)

if image is None:
    print("Could not read input image")
    exit()
    
# initialize hog + svm based face detector
hog_face_detector = dlib.get_frontal_face_detector()

# initialize cnn based face detector with the weights
# cnn_face_detector = dlib.cnn_face_detection_model_v1(args.weights)
cnn_face_detector = dlib.cnn_face_detection_model_v1(weight_path)




#### Applying HOG face detection

start = time.time()

# apply face detection (hog)
faces_hog = hog_face_detector(image, 1)

end = time.time()
print("Execution Time (in seconds) :")
print("HOG : ", format(end - start, '.2f'))

# loop over detected faces
for face in faces_hog:
    x = face.left()
    y = face.top()
    w = face.right() - x
    h = face.bottom() - y

    # draw box over face
    cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)


    
#### Applying CNN face detection

start = time.time()

# apply face detection (cnn)
faces_cnn = cnn_face_detector(image, 1)

end = time.time()
print("CNN : ", format(end - start, '.2f'))

# loop over detected faces
for face in faces_cnn:
    x = face.rect.left()
    y = face.rect.top()
    w = face.rect.right() - x
    h = face.rect.bottom() - y

     # draw box over face
    cv2.rectangle(image, (x,y), (x+w,y+h), (0,0,255), 2)


    
    
#### Display results

# write at the top left corner of the image
# for color identification
img_height, img_width = image.shape[:2]
cv2.putText(image, "HOG", (img_width-50,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (0,255,0), 2)
cv2.putText(image, "CNN", (img_width-50,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (0,0,255), 2)

# display output image
cv2.imshow("face detection with dlib", image)
cv2.waitKey()

# save output image 
cv2.imwrite("cnn_face_detection.png", image)

# close all windows
cv2.destroyAllWindows()

##  Haar Cascade Based  Face  Detection

In [ ]:
# Face RecognitionPlus

import cv2

face_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_frontalface_default.xml') # We load the cascade for the face.
eye_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_eye.xml')
glass_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_eye_tree_eyeglasses.xml')
fullbody_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_fullbody.xml')
upperbody_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_upperbody.xml')
lowerbody_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_lowerbody.xml')
smile_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_smile.xml')
hand_cascade = cv2.CascadeClassifier('HaarCascade/hand.xml')
fist_cascade = cv2.CascadeClassifier('HaarCascade/fist.xml')
hand1_cascade = cv2.CascadeClassifier('HaarCascade/Hand.Cascade.1.xml')
palm_cascade = cv2.CascadeClassifier('HaarCascade/palm.xml')


def detect(gray, frame):
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors= 5)
    '''
    minSize=(30, 30),
    #flags=cv2.cv.CV_HAAR_SCALE_IMAGE
    '''
    
    for (x, y, w, h) in faces:
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, 'This Face!', (x, y), font, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.2, 4)
        for (ex, ey, ew, eh) in eyes:
            cv2.putText(frame, 'eye!', (x+ex, y+ey), font, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
#             cv2.rectangle(roi_color,(ex, ey),(ex+ew, ey+eh), (0, 255, 0), 2)
            cv2.circle(roi_color,(int(ex + ew/2), int(ey+eh/2)), int(ew/2), (0, 255, 0), 2)
#         glass = glass_cascade.detectMultiScale(roi_gray, 1.2, 3)
#         for (ex, ey, ew, eh) in glass:
#             cv2.rectangle(roi_color,(ex, ey),(ex+ew, ey+eh), (0, 255, 0), 1.5)
        smile = smile_cascade.detectMultiScale(roi_gray, 1.2, 25)
        for (sx, sy, sw, sh) in smile:
            cv2.putText(frame, 'lips!', (x+sx, y+sy), font, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#             cv2.rectangle(roi_color,(ex, ey),(ex+ew, ey+eh), (0, 0, 255), 1)
#             cv2.circle(roi_color,(int(sx + sw/2), int(sy+sh/2)), int(sw/2), (0, 0, 255), 1)
            cv2.ellipse(roi_color,(int(sx + sw/2), int(sy+sh/2)),(int(sw/2),int(sh/2)), 0, 0, 360, (0, 0, 255), 1)
#     upbody = upperbody_cascade.detectMultiScale(gray, 1.3, 5)
#     for (ux, uy, uw, uh) in upbody:
#         cv2.rectangle(frame, (ux, uy), (ux+uw, uy+uh), (255, 0, 5), 3)
    
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     palm = palm_cascade.detectMultiScale(gray, 1.3, 5)
#     for (px, py, pw, ph) in palm:
#         cv2.putText(frame, 'palm!', (px, py), font, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
#         cv2.rectangle(frame, (px, py), (px+pw, py+ph), (255, 0, 5), 3)
    
#     hand = fist_cascade.detectMultiScale(gray, 1.3, 5)
#     for (hx, hy, hw, hh) in hand:
#         cv2.putText(frame, 'hand!', (hx, hy), font, 0.8, (255, 0, 255), 2, cv2.LINE_AA)
#         cv2.rectangle(frame, (hx, hy), (hx+hw, hy+hh), (255, 0, 255), 3)
            
    return frame # We return the image with the detector rectangles.

video_capture = cv2.VideoCapture(0) # We turn the webcam on.

while True: # We repeat infinitely (until break):
    _, frame = video_capture.read() # We get the last frame.
#     print(frame)
    frame = cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # We do some colour transformations.
    canvas = detect(gray, frame) # We get the output of our detect function.
    cv2.imshow('Video', canvas) # We display the outputs.
    if cv2.waitKey(1) & 0xFF == ord('q'): # If we type on the keyboard:
        break # We stop the loop.

video_capture.release() # We turn the webcam off.
cv2.destroyAllWindows() # We destroy all the windows inside which the images were displayed.

In [10]:
'''
Based On Real image based Face Detection
'''

import cv2, time, numpy as np
import dlib

weight_path = 'mmod_human_face_detector.dat'

face_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_frontalface_default.xml') # We load the cascade for the face.
eye_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_eye.xml')
glass_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_eye_tree_eyeglasses.xml')
fullbody_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_fullbody.xml')
upperbody_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_upperbody.xml')
lowerbody_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_lowerbody.xml')
smile_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_smile.xml')
hand_cascade = cv2.CascadeClassifier('HaarCascade/hand.xml')
fist_cascade = cv2.CascadeClassifier('HaarCascade/fist.xml')
hand1_cascade = cv2.CascadeClassifier('HaarCascade/Hand.Cascade.1.xml')
palm_cascade = cv2.CascadeClassifier('HaarCascade/palm.xml')



def detect_Haar(gray, frame, SaveFace = False):
    
    # DetectedFace = np.zeros((gray.shape[1], gray.shape[0],3), np.uint8) 
    faceimageshape = (280, 320)
    DetectedFace = np.zeros((faceimageshape[0], faceimageshape[1]), np.uint8) ## Creating blank image
    
    start = time.time()
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors= 5)
    end = time.time()
    for (x, y, w, h) in faces:
        cv2.putText(frame, text = 'Haar Detector' +  format(end - start, '.2f') + 's', 
                    org =(x, y), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.4, 
                    color = (255, 0, 0), thickness = 2, lineType = cv2.LINE_AA)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.2, 4)
        for (ex, ey, ew, eh) in eyes:
            cv2.putText(frame, text = 'Haar eye!', org= (x+ex, y+ey), fontFace = cv2.FONT_HERSHEY_SIMPLEX, 
                        fontScale = 0.25, color = (0, 255, 0), thickness = 1, lineType = cv2.LINE_AA)
            # cv2.rectangle(roi_color,(ex, ey),(ex+ew, ey+eh), (0, 255, 0), 2)
            cv2.circle(roi_color,(int(ex + ew/2), int(ey+eh/2)), int(ew/2), (0, 255, 0), 2)
        # glass = glass_cascade.detectMultiScale(roi_gray, 1.2, 3)
        # for (ex, ey, ew, eh) in glass:
        #     cv2.rectangle(roi_color,(ex, ey),(ex+ew, ey+eh), (0, 255, 0), 1.5)
        smile = smile_cascade.detectMultiScale(roi_gray, 1.2, 25)
        for (sx, sy, sw, sh) in smile:
            cv2.putText(frame, text = 'Haar lips!', org= (x+sx, y+sy), fontFace = cv2.FONT_HERSHEY_SIMPLEX, 
                        fontScale = 0.25, color = (0, 0, 255), thickness = 1, lineType = cv2.LINE_AA)
            # cv2.rectangle(roi_color,(ex, ey),(ex+ew, ey+eh), (0, 0, 255), 1)
            # cv2.circle(roi_color,(int(sx + sw/2), int(sy+sh/2)), int(sw/2), (0, 0, 255), 1)
            cv2.ellipse(roi_color,(int(sx + sw/2), int(sy+sh/2)),(int(sw/2),int(sh/2)), 0, 0, 360, (0, 0, 255), 1)
    # upbody = upperbody_cascade.detectMultiScale(gray, 1.3, 5)
    # for (ux, uy, uw, uh) in upbody:
    #     cv2.rectangle(frame, (ux, uy), (ux+uw, uy+uh), (255, 0, 5), 3)
    # font = cv2.FONT_HERSHEY_SIMPLEX
    # palm = palm_cascade.detectMultiScale(gray, 1.3, 5)
    # for (px, py, pw, ph) in palm:
    #     cv2.putText(frame, 'palm!', (px, py), font, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
    #     cv2.rectangle(frame, (px, py), (px+pw, py+ph), (255, 0, 5), 3)
    # hand = fist_cascade.detectMultiScale(gray, 1.3, 5)
    # for (hx, hy, hw, hh) in hand:
    #     cv2.putText(frame, 'hand!', (hx, hy), font, 0.8, (255, 0, 255), 2, cv2.LINE_AA)
    #     cv2.rectangle(frame, (hx, hy), (hx+hw, hy+hh), (255, 0, 255), 3)
    
    
    
    if len(faces) > 0:
        #cv2.imwrite('saveFace' + str(time.time()) + '.jpg',frame[x:x+w, y:y+h])
        DetectedFace = cv2.resize(gray[x:x+w, y:y+h], dsize = faceimageshape) 
        # print(DetectedFace)
        if SaveFace: cv2.imwrite(filename = 'WebcamExtractedFaces/SaveFace_Haar_' + str(int(time.time())) + '.jpg', img = DetectedFace) ## int time to have less number of images ##smaller image is saved 
        # break
    
    return frame, DetectedFace ## Colored image & subset grey cropped image


def detect_HOG(gray, frame, SaveFace = False):
    faceimageshape = (280, 320)
    DetectedFace = np.zeros((faceimageshape[1], faceimageshape[0]), np.uint8)
    start = time.time()
    # initialize hog + svm based face detector
    hog_face_detector = dlib.get_frontal_face_detector()
    faces = hog_face_detector(gray, 1)
    end = time.time()
    for face in faces:
        x = face.left()
        y = face.top()
        w = face.right() - x
        h = face.bottom() - y
        cv2.putText(frame, text = 'HOG Detector' +  format(end - start, '.2f') + 's', 
                    org =(x, y), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.5, 
                    color = (0, 0, 255), thickness = 2, lineType = cv2.LINE_AA)
        cv2.rectangle(frame, pt1 = (x,y), pt2 = (x+w,y+h), color = (0,0,255), 
                      thickness = 2, lineType = cv2.LINE_AA)
        print((x,y), (x+w,y+h))
        if len(faces) > 0:
            print("WWWWW", (x,x+w), '__', (y,y+h))
            DetectedFace = cv2.resize(gray[int(x):int(x+w), int(y):int(y+h)], dsize = faceimageshape) 
            if SaveFace: cv2.imwrite(filename = 'WebcamExtractedFaces/SaveFace_HOG_' + str(int(time.time())) + '.jpg', img = DetectedFace) ## int time to have less number of images ##smaller image is saved 
            # break
    return frame, DetectedFace


def detect_CNN(gray, frame, SaveFace = False):
    faceimageshape = (280, 320)
    DetectedFace = np.zeros((faceimageshape[0], faceimageshape[1]), np.uint8)
    start = time.time()
    # initialize cnn based face detector with the weights
    cnn_face_detector = dlib.cnn_face_detection_model_v1(weight_path)
    faces = cnn_face_detector(gray, 1)
    end = time.time()
    for face in faces:
        x = face.rect.left()
        y = face.rect.top()
        w = face.rect.right() - x
        h = face.rect.bottom() - y
        
        cv2.putText(frame, text = 'CNN Detector' +  format(end - start, '.2f') + 's', 
                    org =(x, y), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.5, 
                    color = (0, 255, 0), thickness = 2, lineType = cv2.LINE_AA)
        cv2.rectangle(frame, pt1 = (x,y), pt2 = (x+w,y+h), color = (0,255,0), 
                      thickness = 2, lineType = cv2.LINE_AA)
    if len(faces) > 0:
        DetectedFace = cv2.resize(gray[x:x+w, y:y+h], dsize = faceimageshape) 
        if SaveFace: cv2.imwrite(filename = 'WebcamExtractedFaces/SaveFace_CNN_' + str(int(time.time())) + '.jpg', img = DetectedFace) ## int time to have less number of images ##smaller image is saved 
        # break
    return frame, DetectedFace






# ## Haar Cascade ----Mixing is not Done because of the latency
# video_capture = cv2.VideoCapture(0) # We turn the webcam on.
# if(not video_capture.isOpened()):
#     print('Error Camera')
# saveFace = None
# while True: # We repeat infinitely (until break):
# #     _, frame = video_capture.read() # We get the last frame.
# #     print(frame)
#     retval, frame = video_capture.read() # We get the last frame.
#     if retval == False:
#         break # Exit if video has ended
#     elif retval ==True:
        
# #         mask = np.zeros_like(frame)  # init mask
# #         contours = find_contours(frame)
# #         plates, plates_images, mask = find_plate_numbers(frame, contours, mask)
# #         processed_frame = cv2.add(frame, mask)  # Apply the mask to image
# #         cv2.imshow('frame', processed_frame)
        
        
#         ## To Pace Thing up -- almost real time
#         frame = cv2.resize(frame, dsize=None, fx=0.5, fy=0.5) 
#         frame = cv2.flip(frame, 1)
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # We do some colour transformations.
        
#         # print('Frame Shape:',frame.shape) ## Colored Image
#         # print('Gray Shape:', gray.shape)  ## Grey Image
        
#         canvas, grey_face_canvas = detect_Haar(gray, frame, SaveFace = True)
        
#         canvas = cv2.resize(canvas, dsize=None, fx=3, fy=3) ## To increase Size Again
#         cv2.imshow('Video: Haar Cascade Based Face  Detection', canvas)
#         cv2.imshow('Video: Haar Cascade Based Detected  Face', grey_face_canvas)
#         if cv2.waitKey(1) & 0xFF == ord('q'): # If we type on the keyboard:
#             video_capture.release() # We turn the webcam off.
#             cv2.destroyAllWindows() # We destroy all the windows inside which the images were displayed.
#             break # We stop the loop.



## Haar/HOG/CNN based Face Detection
WhichDetectorToUse = 'HOG'

video_capture = cv2.VideoCapture(0)
if(not video_capture.isOpened()):
    print('Error Camera')
while True:
    retval, frame = video_capture.read()
    if retval == False:
        break
    elif retval ==True:
        frame = cv2.resize(frame, dsize=None, fx=0.5, fy=0.5) 
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if WhichDetectorToUse == 'Haar':
            canvas, grey_face_canvas = detect_Haar(gray, frame, SaveFace = True)
        elif WhichDetectorToUse == 'HOG':
            canvas, grey_face_canvas = detect_HOG(gray, frame, SaveFace = True)
        elif WhichDetectorToUse == 'CNN':
            canvas, grey_face_canvas = detect_CNN(gray, frame, SaveFace = True)
        canvas = cv2.resize(canvas, dsize=None, fx=3, fy=3)
        cv2.imshow('Video: HOG Based Face Detection', canvas)
        cv2.imshow('Video: HOG Based Detected Face', grey_face_canvas)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            video_capture.release()
            cv2.destroyAllWindows()
            break


        
# ## Mixed All In Single Frame
# video_capture = cv2.VideoCapture(0)
# while True:
#     _, frame = video_capture.read()
#     frame = cv2.flip(frame, 1)
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
#     canvas = detect_Haar(gray, frame)
#     canvas = detect_HOG(gray, canvas)
#     canvas = detect_CNN(gray, canvas)
#     cv2.imshow('Video: Haar Cascade, HOG, &  CNN Based Face Detection', canvas)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# video_capture.release()
# cv2.destroyAllWindows()








# ## Motion Sensor
# def video (seconds, frameRate):
#     cap = cv2.VideoCapture(0)
#     if(not cap.isOpened()):
#         return "error"

#     # Define the codec and create VideoWriter object
#     fourcc = cv2.cv.CV_FOURCC(*'XVID')
#     name = "media/video/" + time.strftime("%d-%m-%Y_%X")+".avi"
#     out = cv2.VideoWriter(name, fourcc, frameRate, (640,480))
#     program_starts = time.time()
#     result = subprocess.Popen(["ffprobe", name], stdout = subprocess.PIPE, stderr = subprocess.STDOUT, shell=True)
#     nFrames=0
#     while(nFrames<seconds*frameRate):
#         ret, frame = cap.read()
#         if ret==True:
#             out.write(frame)
#             nFrames += 1
#         else:
#             break
#     cap.release()
#     return name 



(96, 105) (171, 179)
(96, 105) (171, 179)
(96, 105) (171, 179)
(96, 105) (171, 179)
(104, 88) (179, 163)
(121, 88) (196, 163)
(128, 94) (190, 156)
(128, 94) (190, 156)
(128, 94) (190, 156)
(128, 94) (190, 156)


In [ ]:
'''
Motion Detection:
https://github.com/RobinDavid/Motion-detection-OpenCV/blob/master/MotionDetector.py :::: Too Many Error To Resolve -----> HEnce Ignnoring
https://www.pyimagesearch.com/2015/05/25/basic-motion-detection-and-tracking-with-python-and-opencv/ ------>>>>>  Implemented
http://www.steinm.com/blog/motion-detection-webcam-python-opencv-differential-images/     ---------->>>> Impplemented
'''
import cv2

def differentialImage(t0, t1, t2):
    d1 = cv2.absdiff(t2, t1)
    d2 = cv2.absdiff(t1, t0)
    return cv2.bitwise_and(d1, d2)

video_capture = cv2.VideoCapture(0)

winName = "Movement Indicator"
cv2.namedWindow(winName, cv2.WINDOW_AUTOSIZE) #,cv2.WINDOW_NORMAL

# Read three images first:
t_minus = cv2.cvtColor(video_capture.read()[1], cv2.COLOR_RGB2GRAY)
t = cv2.cvtColor(video_capture.read()[1], cv2.COLOR_RGB2GRAY)
t_plus = cv2.cvtColor(video_capture.read()[1], cv2.COLOR_RGB2GRAY)

while True:
    cv2.imshow(winName, differentialImage(t_minus, t, t_plus))
    #cv2.imwrite("image.png", diffImg(t_minus, t, t_plus))

    # Read next image
    t_minus = t
    t = t_plus
    t_plus = cv2.cvtColor(video_capture.read()[1], cv2.COLOR_RGB2GRAY)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        video_capture.release()
        cv2.destroyAllWindows()
        break

print('Camera Turning Off')


In [ ]:
'''
Motion Detection ---+ mask + Face Detection Integration (add +20% more area around Face)
'''

import cv2, numpy as np, sys

def differentialImage(t0, t1, t2):
    
    # compute the absolute difference between the current frame and
    frameDelta_d1 = cv2.absdiff(t2, t1)
    thresh_d1 = cv2.threshold(frameDelta_d1, 20, 255, cv2.THRESH_BINARY)[1]
    frameDelta_d2 = cv2.absdiff(t1, t0)
    thresh_d2 = cv2.threshold(frameDelta_d2, 20, 255, cv2.THRESH_BINARY)[1]
    
    # dilate the thresholded image to fill in holes, then find contours on thresholded image
    # Taking a matrix of size 5 as the kernel
    kernel = np.ones((1,1), np.uint8)
    
    thresh_d1 = cv2.dilate(thresh_d1, kernel, iterations=2)
    cnts1 = cv2.findContours(thresh_d1.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #cnts1 = cnts1[0] if imutils.is_cv2() else cnts1[1]
    thresh_d2 = cv2.dilate(thresh_d2, kernel, iterations=2)
    cnts2 = cv2.findContours(thresh_d2.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #cnts2 = cnts2[0] if imutils.is_cv2() else cnts2[1]
    
#     return cv2.bitwise_and(frameDelta_d1, frameDelta_d2)
    return cv2.bitwise_or(thresh_d1, thresh_d2)
#     return cv2.bitwise_or(cnts1, cnts2)



print('Camera Turning On')
video_capture = cv2.VideoCapture(0)
if(not video_capture.isOpened()):
    print('Error Camera')
    sys.exit(1)

# Read three images first:
t_minus = cv2.cvtColor(video_capture.read()[1], cv2.COLOR_RGB2GRAY)
t = cv2.cvtColor(video_capture.read()[1], cv2.COLOR_RGB2GRAY)
t_plus = cv2.cvtColor(video_capture.read()[1], cv2.COLOR_RGB2GRAY)

# t_minus = cv2.GaussianBlur(t_minus, (21, 21), 0)
# t = cv2.GaussianBlur(t, (21, 21), 0)
# t_plus = cv2.GaussianBlur(t_plus, (21, 21), 0)
    
while True:
    retval, frame = video_capture.read()
    if retval == False:
        break
    elif retval ==True:
        
        
        cv2.imshow('Movement Indicator', differentialImage(t_minus, t, t_plus))
        #cv2.imwrite("image.png", diffImg(t_minus, t, t_plus))
        ## Read next image
        t_minus = t
        t = t_plus
        t_plus = cv2.cvtColor(video_capture.read()[1], cv2.COLOR_RGB2GRAY)
        
#         t_minus = cv2.GaussianBlur(t_minus, (21, 21), 0)
#         t = cv2.GaussianBlur(t, (21, 21), 0)
#         t_plus = cv2.GaussianBlur(t_plus, (21, 21), 0)
        
        
        
    
#         frame = cv2.resize(frame, dsize=None, fx=0.5, fy=0.5) 
#         frame = cv2.flip(frame, 1)
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         if WhichDetectorToUse == 'Haar':
#             canvas, grey_face_canvas = detect_Haar(gray, frame, SaveFace = True)
#         elif WhichDetectorToUse == 'HOG':
#             canvas, grey_face_canvas = detect_HOG(gray, frame, SaveFace = True)
#         elif WhichDetectorToUse == 'CNN':
#             canvas, grey_face_canvas = detect_CNN(gray, frame, SaveFace = True)
#         frame = cv2.resize(frame, dsize=None, fx=3, fy=3)
#         cv2.imshow('Video: HOG Based Face Detection', canvas)
#         cv2.imshow('Video: HOG Based Detected Face', grey_face_canvas)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print('Camera Turning Off')
            video_capture.release()
            cv2.destroyAllWindows()
            break

## OpenCv Based Face Detection and Recognition
https://www.hackster.io/mjrobot/real-time-face-recognition-an-end-to-end-project-a10826  
![Path](FaceRecogBlock.png)

#### Test Camera

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height
 
while(True):
    ret, frame = cap.read()
#     frame = cv2.flip(frame, -1) # Flip camera vertically
    frame = cv2.flip(frame, 1) # Flip camera horizontally
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('frame', frame)
    cv2.imshow('gray', gray)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27: # press 'ESC' to quit
        break

cap.release()
cv2.destroyAllWindows()


#### Face Detection

In [122]:
import numpy as np
import cv2

# faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')
face_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
cap.set(3,640) # set Width
cap.set(4,480) # set Height

while True:
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors= 5, minSize=(20, 20))   

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]  

    cv2.imshow('video',img)

    k = cv2.waitKey(30) & 0xff
    if k == 27: # press 'ESC' to quit
        break

cap.release()
cv2.destroyAllWindows()


#### Data Gathering

In [3]:
import cv2
import os,  sys
import pandas as pd
import glob, time


def CapturePersonFaceImages(dirToSaveImageIn, perFaceId):
    '''
    
    '''
    
    cam = cv2.VideoCapture(0)
    cam.set(3, 640) # set video width
    cam.set(4, 480) # set video height
    
    # face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    face_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_frontalface_default.xml')

    print("\n [INFO] Initializing face capture. Look at the camera and wait ...")
    # Initialize individual sampling face count
    AllFiles = glob.glob(dirToSaveImageIn + 'User.{}.*.jpg'.format(perFaceId))
    ini_count = len(AllFiles) #0 if len(AllFiles) == 0 else 1
    count = 0
    
    while 0xFF == 255:
        ret, img = cam.read()
        cv2.putText(img, text = 'Press any button when ready!', org= (15, 60), fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                fontScale = 1, color = (0, 0, 255), thickness = 1, lineType = cv2.LINE_AA)
        cv2.imshow('image', img)
        time.sleep(0.1)
        if  cv2.waitKey(100) & 0xff != 255:
            break
    
    while(True):
        ret, img = cam.read()
        img = cv2.flip(img, 1)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        cv2.putText(img, text = 'Please look at the camera!', org= (15, 30), fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale = 1, color = (0, 0, 255), thickness = 1, lineType = cv2.LINE_AA)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors= 5, minSize=(20, 20))
        for (x,y,w,h) in faces:
            print((x,y))
            cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
            cv2.imshow('image', img) ## frame  stops if  face is not visbile
        cv2.imshow('image', img) ## frame keep on showing
        if len(faces) > 0:
            # DetectedFace = cv2.resize(gray[x:x+w, y:y+h], dsize = faceimageshape) 
            count += 1
            print('Image {} Taken'.format(str(count)))
            time.sleep(0.1)
            # Save the captured image into the datasets folder
            cv2.imwrite(dirToSaveImageIn + 'User.' + str(perFaceId) + '.' + str(ini_count+count) + ".jpg", gray[y:y+h,x:x+w])
        
#         if cv2.waitKey(100) & 0xff != 255:
            
        k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break
        elif count >= 30: # Take 30 face sample and stop video
             break
            
    # Do a bit of cleanup
    print("\n [INFO] Exiting Program and cleanup stuff")
    cam.release()
    cv2.destroyAllWindows()



# Path for face image database
path = 'dataset/'
personIdMapFile_path = 'MappingFaceToId.csv'


# For each person, enter one numeric face id
personFirName = input('\n Enter user First name: ')# Hit Enter<return> at the end. #input('\n enter user id end press <return> ==>  ')
personLasName = input('\n Enter user Last name: ')
personName = '_'.join([personFirName, personLasName, 'id0'])
#print(personName)

if os.path.exists(personIdMapFile_path):
    perMap = pd.read_csv(personIdMapFile_path)
    if personName in list(perMap['PersonName']):
        ## if person name is already present add more data to it
        print('\nThis Person Nane is already present in the DB')
        action = input('Type "SamePerson" to add more data to this person profile or "Exit" to exit or "NewPerson" to add information for a new person having same name')
        if action == 'SamePerson':
            face_id = perMap.loc[perMap['PersonName']==personName, 'ID'][0]
            CapturePersonFaceImages(path, face_id)
        elif action == 'NewPerson':
            pVal = sum(perMap['PersonName'] == personName) + 1
            personName = '_'.join([personName.split('_')[0], personName.split('_')[1], 'id'+str(pVal)])
            face_id = perMap.shape[0] + 1
            perMap = perMap.append({'ID':face_id, 'PersonName':personName}, ignore_index=True)
            CapturePersonFaceImages(path, face_id)
        else:
            sys.exit('User Selected the exit command, when name of the person was matching.')
    else:
        ## adding impformation about new person
        print('\nAdding facial Information of a new person')
        face_id = max(perMap['ID']) + 1 ## other just get the length of the df 
        perMap = perMap.append({'ID':face_id, 'PersonName':personName}, ignore_index=True)
        CapturePersonFaceImages(path, face_id)
else:
    ## This is the first iteration creating mmapping dataset
    perMap = pd.DataFrame({'ID':1, 'PersonName':personName}, columns=['ID', 'PersonName'],  index=[0])
    CapturePersonFaceImages(path, 1)

perMap.to_csv(personIdMapFile_path, index=False)
perMap.tail(5)


 Enter user First name: aakanksha

 Enter user Last name: asd

Adding facial Information of a new person

 [INFO] Initializing face capture. Look at the camera and wait ...
(166, 338)
Image 1 Taken
(194, 335)
Image 2 Taken
(194, 327)
Image 3 Taken
(171, 183)
Image 4 Taken
(169, 310)
Image 5 Taken
(152, 282)
Image 6 Taken
(146, 281)
Image 7 Taken
(90, 284)
Image 8 Taken
(66, 281)
Image 9 Taken
(143, 282)
Image 10 Taken
(126, 275)
Image 11 Taken
(128, 253)
Image 12 Taken
(114, 225)
Image 13 Taken
(90, 200)
Image 14 Taken
(98, 186)
Image 15 Taken
(120, 137)
Image 16 Taken
(125, 120)
Image 17 Taken
(186, 90)
Image 18 Taken
(198, 78)
Image 19 Taken
(202, 87)
Image 20 Taken
(204, 87)
Image 21 Taken
(205, 96)
Image 22 Taken
(212, 107)
Image 23 Taken
(210, 110)
Image 24 Taken
(216, 121)
Image 25 Taken
(211, 124)
Image 26 Taken
(217, 131)
Image 27 Taken
(221, 137)
Image 28 Taken
(225, 147)
Image 29 Taken
(223, 148)
Image 30 Taken

 [INFO] Exiting Program and cleanup stuff


,ID,PersonName
0,1,Mohit_Rajput_id0
1,2,aman_rawka_id0
2,3,Mohit__id0
3,4,aakanksha_asd_id0


In [3]:
cv2.waitKey(100) & 0xff

255

#### Trainer

In [4]:
import cv2
import numpy as np
from PIL import Image
import os

# Path for face image database
path = 'dataset'
model_Saving_Path = 'trainer'

recognizer = cv2.face.LBPHFaceRecognizer_create()  ## requires to install: pip3 install opencv-contrib-python
detector = cv2.CascadeClassifier('HaarCascade/haarcascade_frontalface_default.xml')

# function to get the images and label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
#         id = int(os.path.split(imagePath)[-1].split(".")[-2])
        faces = detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)
    return faceSamples,ids

print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))

# Save the model into trainer/trainer.yml
recognizer.write(model_Saving_Path + '/trainer.yml') # recognizer.save() worked on Mac, but not on Pi

# Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))



 [INFO] Training faces. It will take a few seconds. Wait ...

 [INFO] 4 faces trained. Exiting Program


In [ ]:
# import opencv-contrib-python
# cv2.FisherFaceRecognizer_create()

#### Recognizer

In [116]:
personIdMapFile_path = 'MappingFaceToId.csv'
perDf = pd.read_csv(personIdMapFile_path)
['None'] + list(perDf['PersonName'])

['None', 'Mohit_Rajput_id0', 'aman_rawka_id0']

In [5]:
import cv2
import numpy as np
import os, pandas as pd

recognizer = cv2.face.LBPHFaceRecognizer_create()
## Three Recognizer Available with OpenCV
# Eigen Face  Recognizer
# Fisher Face  Recognizer
# LBPH Face Recognizer

personIdMapFile_path = 'MappingFaceToId.csv'
perDf = pd.read_csv(personIdMapFile_path)
recognizer.read('trainer/trainer.yml')
cascadePath = "HaarCascade/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

font = cv2.FONT_HERSHEY_SIMPLEX

#iniciate id counter
id = 0

# names related to ids: example ==> Marcelo: id=1,  etc
names = ['None'] + list(perDf['PersonName']) 

# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height

# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)

while True:
    ret, img =cam.read()
    img = cv2.flip(img, 1) # Flip horizontally
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors = 5, minSize = (int(minW), int(minH)))

    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

        # Check if confidence is less them 100 ==> "0" is perfect match 
        if (confidence < 100):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2) ## Displaying Name
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1) ## Displaying Confidence
    
    cv2.imshow('camera',img) 

    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break

# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()



 [INFO] Exiting Program and cleanup stuff


#### Code Snippet -- Plain Text

In [ ]:
faces = faceCascade.detectMultiScale(
        gray,     
        scaleFactor=1.2,
        minNeighbors=5,     
        minSize=(20, 20)
    )


In [6]:
### Saving a Video
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.flip(frame,1)

        # write the flipped frame
        out.write(frame)

        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

## OpenCV
https://www.superdatascience.com/opencv-face-recognition/  
https://github.com/informramiz/opencv-face-recognition-python -- >  better

#### Libraries

In [ ]:
#OpenCV module
import cv2
#os module for reading training data directories and paths
import os
#numpy to convert python lists to numpy arrays as it is needed by OpenCV face recognizers
import numpy as np

#### Prepare training data

In [ ]:
#there is no label 0 in our training data so subject name for index/label 0 is empty
subjects = ["", "Ramiz Raja", "Elvis Presley"]
subject = ["", "Aaron_Peirsol", "Abdullah_Gul", ""]

#### Data Preparation for Face Recognition

In [ ]:
#function to detect face using OpenCV
def detect_face(img):
    #convert the test image to gray scale as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #load OpenCV face detector, I am using LBP which is fast
    #there is also a more accurate but slow: Haar classifier
    # face_cascade = cv2.CascadeClassifier('opencv-files/lbpcascade_frontalface.xml')
    face_cascade = cv2.CascadeClassifier('HaarCascade/haarcascade_frontalface_default.xml')

    #let's detect multiscale images(some images may be closer to camera than others)
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);

    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None

    #under the assumption that there will be only one face,
    #extract the face area
    (x, y, w, h) = faces[0]

    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

In [ ]:
#this function will read all persons' training images, detect face from each image
#and will return two lists of exactly same size, one list 
#of faces and another list of labels for each face
def prepare_training_data(data_folder_path):
    #------STEP-1--------
    #get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)

    #list to hold all subject faces
    faces = []
    #list to hold labels for all subjects
    labels = []

    #let's go through each directory and read images within it
    for dir_name in dirs:

        #our subject directories start with letter 's' so
        #ignore any non-relevant directories if any
        if not dir_name.startswith("s"):
            continue;

    #------STEP-2--------
    #extract label number of subject from dir_name
    #format of dir name = slabel
    #, so removing letter 's' from dir_name will give us label
    label = int(dir_name.replace("s", ""))

    #build path of directory containing images for current subject subject
    #sample subject_dir_path = "training-data/s1"
    subject_dir_path = data_folder_path + "/" + dir_name

    #get the images names that are inside the given subject directory
    subject_images_names = os.listdir(subject_dir_path)

    #------STEP-3--------
    #go through each image name, read image, 
    #detect face and add face to list of faces
    for image_name in subject_images_names:

        #ignore system files like .DS_Store
        if image_name.startswith("."):
            continue;

    #build image path
    #sample image path = training-data/s1/1.pgm
    image_path = subject_dir_path + "/" + image_name

    #read image
    image = cv2.imread(image_path)

    #display an image window to show the image 
    cv2.imshow("Training on image...", image)
    cv2.waitKey(100)

    #detect face
    face, rect = detect_face(image)

    #------STEP-4--------
    #for the purpose of this tutorial
    #we will ignore faces that are not detected
    if face is not None:
        #add face to list of faces
        faces.append(face)
        #add label for this face
        labels.append(label)

    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()

    return faces, labels

In [ ]:
#let's first prepare our training data
#data will be in two lists of same size
#one list will contain all the faces
#and the other list will contain respective labels for each face
print("Preparing data...")
faces, labels = prepare_training_data("training-data")
print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

#### Train Face Recognizer

In [ ]:
#create our LBPH face recognizer 
face_recognizer = cv2.face.createLBPHFaceRecognizer()

#or use EigenFaceRecognizer by replacing above line with 
#face_recognizer = cv2.face.createEigenFaceRecognizer()

#or use FisherFaceRecognizer by replacing above line with 
#face_recognizer = cv2.face.createFisherFaceRecognizer()

In [ ]:
#train our face recognizer of our training faces
face_recognizer.train(faces, np.array(labels))

#### Prediction

In [ ]:
#function to draw rectangle on image 
#according to given (x, y) coordinates and 
#given width and heigh
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

#function to draw text on give image starting from
#passed (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [ ]:
#this function recognizes the person in image passed
#and draws a rectangle around detected face with name of the 
#subject
def predict(test_img):
#make a copy of the image as we don't want to change original image
img = test_img.copy()
#detect face from the image
face, rect = detect_face(img)

#predict the image using our face recognizer 
label= face_recognizer.predict(face)
#get name of respective label returned by face recognizer
label_text = subjects[label]
 
#draw a rectangle around face detected
draw_rectangle(img, rect)
#draw name of predicted person
draw_text(img, label_text, rect[0], rect[1]-5)
 
return img

In [ ]:
print("Predicting images...")

#load test images
test_img1 = cv2.imread("test-data/test1.jpg")
test_img2 = cv2.imread("test-data/test2.jpg")

#perform a prediction
predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)
print("Prediction complete")

#display both images
cv2.imshow(subjects[1], predicted_img1)
cv2.imshow(subjects[2], predicted_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

##  Open  CV YOLO
https://www.arunponnusamy.com/yolo-object-detection-opencv-python.html  
https://github.com/zhreshold/mxnet-yolo  
https://pjreddie.com/darknet/yolo/  

In [ ]:
# import required packages
import cv2
import argparse
import numpy as np

# handle command line arguments
ap = argparse.ArgumentParser()
ap.add_argument('-i', '--image', required=True,
                help = 'path to input image')
ap.add_argument('-c', '--config', required=True,
                help = 'path to yolo config file')
ap.add_argument('-w', '--weights', required=True,
                help = 'path to yolo pre-trained weights')
ap.add_argument('-cl', '--classes', required=True,
                help = 'path to text file containing class names')
args = ap.parse_args()

#### Preparing Data

In [ ]:
# read input image
image = cv2.imread(args.image)

Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

# read class names from text file
classes = None
with open(args.classes, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# read pre-trained model and config file
net = cv2.dnn.readNet(args.weights, args.config)

# create input blob 
blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

# set input blob for the network
net.setInput(blob)

#### Output layer and bounding box

In [ ]:
# function to get the output layer names 
# in the architecture
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

#### Running inference

In [ ]:
# run inference through the network
# and gather predictions from output layers
outs = net.forward(get_output_layers(net))

# initialization
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4

# for each detetion from each output layer 
# get the confidence, class id, bounding box params
# and ignore weak detections (confidence < 0.5)
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

In [ ]:
#### Non-max suppression

In [ ]:
# apply non-max suppression
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

# go through the detections remaining
# after nms and draw bounding box
for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    
    draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

# display output image    
cv2.imshow("object detection", image)

# wait until any key is pressed
cv2.waitKey()
    
 # save output image to disk
cv2.imwrite("object-detection.jpg", image)

# release resources
